# µNotebook Stress Test

This notebook contains **MicroPython-style** code snippets designed to stress-test a notebook frontend: large outputs, unicode, ANSI color, errors, long lines, binary-ish data, and incremental logging.

- You can run cells individually to verify streaming, truncation, and rendering.
- The code is written to be compatible with standard Python syntax that MicroPython also supports (adjust imports like `utime` vs `time` as needed by your runtime).


In [ ]:
# 1) Hello + quick sanity check (UTF-8, symbols)
print('Hello from MicroPython stress test!')
print('Greek mu: µ  |  Emoji: 🚀  |  Accents: café  |  RTL: مرحبا')


In [ ]:
# 2) Rapid 1000-line output (checks line buffering & scroll performance)
for i in range(1000):
    print('line', i)


In [ ]:
# 3) Progress stream with carriage returns (\r) to test in-place updates
try:
    import utime as time
    sleep = utime.sleep
except ImportError:
    try:
      import time
      sleep = time.sleep
    except ImportError:
      from pybricks.tools import wait
      sleep = wait

for i in range(0, 101):
    bar = '#' * (i // 2)
    s = '\rProgress: {0:3d}% |{1:<50}|'.format(i, bar)
    print(s, end='')
    sleep(0.01)
print('\nDone.')


In [ ]:
# 4) Very long line (10k+ characters) to test wrapping/truncation
long_line = 'A' * 10_000
print(long_line)
print('Length:', len(long_line))


In [ ]:
# 5) ANSI color / style sequences (if your UI supports them)
RESET='\x1b[0m'
BOLD='\x1b[1m'
RED='\x1b[31m'
GREEN='\x1b[32m'
YELLOW='\x1b[33m'
BLUE='\x1b[34m'
print(BOLD + 'Bold line' + RESET)
print(RED + 'Red' + RESET, GREEN + 'Green' + RESET, YELLOW + 'Yellow' + RESET, BLUE + 'Blue' + RESET)
for i in range(6):
    print('\x1b[3%dmColor %d%s' % (i, i, RESET))


In [ ]:
# 6) Unicode torture: combining marks, wide glyphs, RTL, emoji sequences
samples = [
    'Combining: A\u0301 A\u0323 A\u0301\u0323',
    'CJK: 你好，世界',
    'RTL (Arabic): العربية مرحبا',
    'Emoji ZWJ: 👩\u200d💻 👨\u200d🚀',
    'Skin tones: 👍👍🏻👍🏼👍🏽👍🏾👍🏿',
    'Mu: µ — microfarad µF'
]
for s in samples:
    print(s)


In [ ]:
# 7) Binary-like output (NULs and non-printables) using repr to stay text-safe
buf = bytes([i % 256 for i in range(1024)])
print('bytes len:', len(buf))
print(repr(buf))

# Also write some control codes directly to test your renderer
ctrls = b'CTRL-B:\x02 CTRL-C:\x03 NUL:\x00 END' 
try:
    import sys
    sys.stdout.write(ctrls.decode('latin1'))
    sys.stdout.write('\n')
except Exception as e:
    print('write error:', e)


In [ ]:
# 8) Intentional exceptions to test traceback rendering and stderr capture
def boom():
    raise ValueError('Intentional error for traceback test')

try:
    boom()
except Exception as exc:
    import sys
    print('Caught:', exc)
    # Now raise unhandled to see a raw traceback
    raise


In [ ]:
# 9) Large JSON-ish output to test pretty-print and folding
try:
    import ujson as json
except ImportError:
    import json

data = {
    'items': [ {'i': i, 'txt': 'x'*200 } for i in range(500) ],
    'meta': {'created': 'now', 'µ': 'mu', 'note': 'stress-test'}
}
print(json.dumps(data)[:5000])  # print only first 5k chars


In [ ]:
# 10) Streaming logs with flush to test incremental delivery in your UI
try:
    import utime as time
except ImportError:
    import time
for i in range(50):
    print('tick', i)
    try:
        import sys
        sys.stdout.flush()
    except Exception:
        pass
    time.sleep(0.02)


In [ ]:
# 11) Explicit stderr writes if your kernel/bridge routes them distinctly
try:
    import sys
    sys.stderr.write('This is stderr line 1\n')
    sys.stderr.write('This is stderr line 2\n')
except Exception as e:
    print('stderr not available:', e)


In [ ]:
# 12) Longer runtime with intermittent output (cancel-friendly)
try:
    import utime as time
except ImportError:
    import time
for i in range(10):
    print('chunk', i, '...' )
    for j in range(1000):
        pass  # burn CPU briefly
    time.sleep(0.1)
print('long runtime section finished')
